# Transak iteration 1
Feature Engineering with TensorFlow using old API tf.feature_column

## I used knowledge from those courses 
- GCP Proffesional Machine Learning Engineer course 4 Feature Engineering, modules 3,5,6,7,8,9
- Machine Learning Mastery course from Adam Dobrakowski

## Assumptions
1. Floats change into categorical bins (MLE 4.3.1) - quntiles with same number of examples per bin? - perhaps in next iteration with Keras or tf.Transform. Now I need fast next level from benchmark model.
2. Cyclic transformation of dates should allow better identify Abos
3. Environment installation:
   - anaconda: Python 3.9
   - pip: db-dtypes==1.0.5, google-cloud-bigquery==3.9.0, pandas==1.5.3, numpy==1.23.5, tensorflow==2.12.1 - this setup supports old tf.feature_column API and Keras DenseFeatures layer.

# Development

## Imports

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import feature_column as fc

print(f"BigQuery client library version: {bigquery.__version__}")
print(f"Tensorflow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Keras version: {tf.keras.__version__}")

BigQuery client library version: 3.9.0
Tensorflow version: 2.12.1
Pandas version: 1.5.3
Keras version: 2.12.0


## Data load

In [2]:
bq = bigquery.Client(project="af-finanzen")
query = """
  WITH EDA AS (
    SELECT
        tid
      , type
      --AF20250604 I need Cyclic because of minmax normalization , UNIX_SECONDS(started) started -- TF works with numbers or strings, not with timestamps
      --AF20250604 I need Cyclic because of minmax normalization , UNIX_SECONDS(first_started) first_started -- this should be part of feature engineering or ingestion ale update?
      --AF20250604 I need Cyclic because of minmax normalization   -- if this is here, then it is easier do train test eval split because here it is done on all data,
      --AF20250604 I need Cyclic because of minmax normalization   -- an not on train or test or validation separately where it would be different. Same for min max and so on.. 
      , started
      , first_started
      , LOWER(description) description -- i1_eda_conclusion_2
      , amount
      , fee
      , currency
      , state
      , account
      , CASE
          WHEN i1_true_label = 'PK Prezenty' THEN 'PK Rest'
          WHEN i1_true_label = 'Apt' THEN 'PK Kasia'
          ELSE i1_true_label
        END AS i1_true_label
    FROM `af-finanzen.monatsabschluss.revolut_abrechnung`
    WHERE
      type != "FEE" -- i1_eda_conclusion_1
    ORDER BY started
  ),
  LABEL_INT AS (
    SELECT
      *
      , DENSE_RANK() OVER(ORDER BY i1_true_label) - 1 AS i1_true_label_id
    FROM EDA
  ),
  SPLIT_SET AS (
    SELECT
        tid
      , i1_true_label
      , CASE
          WHEN ABS(MOD(tid, 10)) < 8 THEN 'train'
          WHEN ABS(MOD(tid, 10)) = 8 THEN 'validation'
          WHEN ABS(MOD(tid, 10)) = 9 THEN 'test'
          -- WHEN ABS(MOD(tid, 100)) < 70 THEN 'train'
          -- WHEN ABS(MOD(tid, 100)) BETWEEN 70 AND 85 THEN 'validation'
          -- WHEN ABS(MOD(tid, 100)) BETWEEN 86 AND 100 THEN 'test'
          ELSE "unknown"
        END AS split_set
    FROM EDA
    GROUP BY
      i1_true_label, tid
  )
  SELECT
    LABEL_INT.* EXCEPT(tid, i1_true_label)
    , SPLIT_SET.split_set
  FROM LABEL_INT
  JOIN SPLIT_SET
  ON LABEL_INT.tid = SPLIT_SET.tid
"""
df = bq.query(query).to_dataframe()

Back mapping

In [3]:
query = """
  WITH EDA AS (
    SELECT
        CASE
          WHEN i1_true_label = 'PK Prezenty' THEN 'PK Rest'
          WHEN i1_true_label = 'Apt' THEN 'PK Kasia'
          ELSE i1_true_label
        END AS i1_true_label
    FROM `af-finanzen.monatsabschluss.revolut_abrechnung`
    WHERE
      type != "FEE" -- i1_eda_conclusion_1
    ORDER BY started
  )
  SELECT
    DISTINCT i1_true_label,
    DENSE_RANK() OVER (ORDER BY i1_true_label) - 1 AS i1_true_label_id
  FROM EDA
  ORDER BY i1_true_label
"""
rev_mapping_df = bq.query(query).to_dataframe()
rev_mapping_df

,i1_true_label,i1_true_label_id
0,Exchange,0
1,PK Abo,1
2,PK Artur,2
3,PK Auto,3
4,PK Kasia,4
5,PK Leben,5
6,PK Maja,6
7,PK Medic,7
8,PK Reisen,8
9,PK Rest,9


# Cyclic transformations

In [4]:
df['started_day'] = df.started.dt.day
df['started_day_sin'] = np.sin((df.started_day-1)*(2.*np.pi/31))
df['started_day_cos'] = np.cos((df.started_day-1)*(2.*np.pi/31))
df['started_month'] = df.started.dt.month
df['started_month_sin'] = np.sin((df.started_month-1)*(2.*np.pi/12))
df['started_month_cos'] = np.cos((df.started_month-1)*(2.*np.pi/12))
# can not by normalized df['started_year'] = df.started.dt.year
df.drop(columns=['started'], inplace=True)

In [5]:
df['first_started_day'] = df.first_started.dt.day
df['first_started_day_sin'] = np.sin((df.first_started_day-1)*(2.*np.pi/31))
df['first_started_day_cos'] = np.cos((df.first_started_day-1)*(2.*np.pi/31))
df['first_started_month'] = df.first_started.dt.month
df['first_started_month_sin'] = np.sin((df.first_started_month-1)*(2.*np.pi/12))
df['first_started_month_cos'] = np.cos((df.first_started_month-1)*(2.*np.pi/12))
df['first_started_year'] = df.first_started.dt.year
df.drop(columns=['first_started'], inplace=True)

# Train, test, validation split

In [6]:
train_df = df[df['split_set'] == 'train'].drop(columns=['split_set'])
test_df = df[df['split_set'] == 'test'].drop(columns=['split_set'])
val_df = df[df['split_set'] == 'validation'].drop(columns=['split_set'])
print(df.shape, 'all examples and columns')
print(len(train_df), 'train examples')
print(len(test_df), 'test examples')
print(len(val_df), 'validation examples')

(2581, 22) all examples and columns
2093 train examples
251 test examples
237 validation examples


## Transform Pandas DF into Tensorflow Dataset

In [7]:
def df2dataset(df: pd.DataFrame, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop("i1_true_label_id").astype(np.int64)
    for col in df.columns:
        if isinstance(df[col].dtype, pd.Int64Dtype):
            # It's often safest to convert feature columns to float32,
            # as it naturally handles potential missing values if you need them.
            # also solves ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).
            print(f"Converting feature column '{col}' from Int64 to float32.")
            df[col] = df[col].astype(np.float32)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df), seed=42)
    # ds.map(lambda...) M3.2
    ds = ds.batch(batch_size=batch_size)
    return ds

In [8]:
train_ds = df2dataset(train_df)
val_ds = df2dataset(val_df, shuffle=True)
for feature_batch, label_batch in train_ds.take(1):
    print(f'List of feature names: {list(feature_batch.keys())}')
    print(f'Batch of type: {feature_batch["type"]}')
    print(f'Batch of labels: {label_batch}')

List of feature names: ['type', 'description', 'amount', 'fee', 'currency', 'state', 'account', 'started_day', 'started_day_sin', 'started_day_cos', 'started_month', 'started_month_sin', 'started_month_cos', 'first_started_day', 'first_started_day_sin', 'first_started_day_cos', 'first_started_month', 'first_started_month_sin', 'first_started_month_cos', 'first_started_year']
Batch of type: [b'TOPUP' b'CARD_PAYMENT' b'CARD_PAYMENT' b'TOPUP' b'CARD_PAYMENT'
 b'CARD_PAYMENT' b'TOPUP' b'CARD_PAYMENT' b'CARD_PAYMENT' b'CARD_PAYMENT'
 b'TOPUP' b'CARD_PAYMENT' b'CARD_PAYMENT' b'CARD_PAYMENT' b'CARD_PAYMENT'
 b'TOPUP' b'TOPUP' b'TOPUP' b'CARD_PAYMENT' b'CARD_PAYMENT' b'TOPUP'
 b'CARD_PAYMENT' b'TOPUP' b'TOPUP' b'TOPUP' b'CARD_PAYMENT'
 b'CARD_PAYMENT' b'TOPUP' b'CARD_PAYMENT' b'TRANSFER' b'CARD_PAYMENT'
 b'TOPUP']
Batch of labels: [12  5  8 12  5  5 12  5  5  6 12 13 11  2  5 12 12 12  2  2 12  5 12 12
 12 13  5 12  2  5  5 12]


# Feature Columns

In [9]:
feature_columns = []

Numerical columns

In [10]:
numerical_col_names_normalized = ['started_day_sin', 'started_day_cos', 'started_month_sin', 'started_month_cos', 'first_started_day_sin', 'first_started_day_cos', 'first_started_month_sin', 'first_started_month_cos']
numerical_col_names_to_normalize = ['amount', 'fee', 'started_day', 'started_month', 'first_started_day', 'first_started_month', 'first_started_year']

In [11]:
for feature_name in numerical_col_names_normalized:
    feature_columns.append(fc.numeric_column(feature_name))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [12]:
STATS = {}
for feature in numerical_col_names_to_normalize:
    min_val = train_df[feature].min()
    max_val = train_df[feature].max()
    
    STATS[feature] = {
        'min': min_val,
        'max': max_val
    }

def get_scaling_fn(feature: str):
    def minmax(x):
        mini = STATS[feature]['min']
        maxi = STATS[feature]['max']
        return (x-mini)/(maxi-mini + 1e-7)
    return minmax

In [13]:
for feature_name in numerical_col_names_to_normalize:
    scaling_fn = get_scaling_fn(feature_name)
    feature_columns.append(fc.numeric_column(feature_name, normalizer_fn=scaling_fn))

Categorical columns

In [14]:
categorical_columns = ['type', 'description', 'currency', 'state', 'account']
tobe_crossed = []
for feature_name in categorical_columns:
    vocabulary = df[feature_name].unique()
    feature_column = fc.categorical_column_with_vocabulary_list(feature_name, vocabulary, num_oov_buckets=1)
    feature_columns.append(fc.indicator_column(feature_column))
    if feature_name in ['type', 'description']:
        tobe_crossed.append(feature_column)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


Crossed features

In [15]:
type_description_cross = fc.crossed_column(tobe_crossed, hash_bucket_size=100)
type_description_cross = fc.indicator_column(type_description_cross)
feature_columns.append(type_description_cross)

Instructions for updating:
Use `tf.keras.layers.experimental.preprocessing.HashedCrossing` instead for feature crossing when preprocessing data to train a Keras model.


# Model

In [17]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(80, input_dim=20, activation='relu'),
    layers.Dense(40, activation='relu'),
    layers.Dense(14, activation='softmax', name='i1_pred_label_id'),
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_ds, validation_data=val_ds, epochs=8)


Epoch 1/8
66/66 [==============================] - 1s 7ms/step - loss: 1.9939 - accuracy: 0.4219 - val_loss: 1.5017 - val_accuracy: 0.5063
Epoch 2/8
66/66 [==============================] - 0s 3ms/step - loss: 1.2038 - accuracy: 0.6574 - val_loss: 1.0664 - val_accuracy: 0.7215
Epoch 3/8
66/66 [==============================] - 0s 3ms/step - loss: 0.7925 - accuracy: 0.7941 - val_loss: 0.8037 - val_accuracy: 0.7932
Epoch 4/8
66/66 [==============================] - 0s 3ms/step - loss: 0.5616 - accuracy: 0.8614 - val_loss: 0.6817 - val_accuracy: 0.8101
Epoch 5/8
66/66 [==============================] - 0s 3ms/step - loss: 0.4215 - accuracy: 0.8877 - val_loss: 0.6052 - val_accuracy: 0.8228
Epoch 6/8
66/66 [==============================] - 0s 3ms/step - loss: 0.3267 - accuracy: 0.9226 - val_loss: 0.5608 - val_accuracy: 0.8565
Epoch 7/8
66/66 [==============================] - 0s 3ms/step - loss: 0.2629 - accuracy: 0.9422 - val_loss: 0.5130 - val_accuracy: 0.8650
Epoch 8/8
66/66 [==========